<a href="https://colab.research.google.com/github/TrishankV/Ghibli_styler/blob/main/Attmept1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/ghibli_frames.zip -d /content/dataset/
!unzip /content/drive/MyDrive/coco_subset.zip -d /content/dataset/


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/dataset/ghibli_frames/frame_3151.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_0470.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_6001.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_5508.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_2267.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_3179.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_4616.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_4170.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_2501.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_6767.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_1008.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_0316.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/frame_6773.jpg  
  inflating: /content/dataset/dataset/ghibli_frames/

In [5]:
!unzip /content/drive/MyDrive/datasets/flickr30k-images.zip -d /content/dataset/trainA


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/trainA/flickr30k-images/434938585.jpg  
  inflating: /content/dataset/trainA/__MACOSX/flickr30k-images/._434938585.jpg  
  inflating: /content/dataset/trainA/flickr30k-images/3260088697.jpg  
  inflating: /content/dataset/trainA/__MACOSX/flickr30k-images/._3260088697.jpg  
  inflating: /content/dataset/trainA/flickr30k-images/4857774876.jpg  
  inflating: /content/dataset/trainA/__MACOSX/flickr30k-images/._4857774876.jpg  
  inflating: /content/dataset/trainA/flickr30k-images/4410008455.jpg  
  inflating: /content/dataset/trainA/__MACOSX/flickr30k-images/._4410008455.jpg  
  inflating: /content/dataset/trainA/flickr30k-images/2931392634.jpg  
  inflating: /content/dataset/trainA/__MACOSX/flickr30k-images/._2931392634.jpg  
  inflating: /content/dataset/trainA/flickr30k-images/3066429707.jpg  
  inflating: /content/dataset/trainA/__MACOSX/flickr30k-images/._3066429707.jpg  
  inflating: /content/dataset/tra

In [6]:
!mkdir -p /content/dataset/trainA
!mkdir -p /content/dataset/trainB


In [8]:
# Move Flickr30k images to trainA
# !mv /content/dataset/trainA/flickr30k-images/* /content/dataset/trainA/

# Move Ghibli frames to trainB
!mv /content/dataset/dataset/ghibli_frames/* /content/dataset/trainB/


In [12]:
!pip install tensorflow-gpu
!pip install pillow
!pip install matplotlib

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [13]:
!pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import itertools
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [26]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])




In [2]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
%cd pytorch-CycleGAN-and-pix2pix


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516
Receiving objects: 100% (2516/2516), 8.20 MiB | 12.86 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix


In [3]:
!mkdir -p datasets/ghibli2real
!cp -r /content/dataset/trainB datasets/ghibli2real/trainA
!cp -r /content/dataset/trainA/flickr30k-images datasets/ghibli2real/trainB


In [31]:
# prompt: show me where ghibli2real is

!ls pytorch-CycleGAN-and-pix2pix/datasets/


ls: cannot access 'pytorch-CycleGAN-and-pix2pix/datasets/': No such file or directory


In [7]:
!pip install dominate


In [14]:
!git clone https://github.com/taki0112/UGATIT
%cd UGATIT


Cloning into 'UGATIT'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 5.68 MiB | 9.77 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/pytorch-CycleGAN-and-pix2pix/UGATIT


In [15]:
!python main.py --dataset /content/dataset --phase train --light True


2024-07-26 18:47:36.159308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 18:47:36.517240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 18:47:36.614420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 18:47:37.191722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 18:47:40.840562: W tensorflow/compiler/tf2

In [16]:
!python main.py --dataset /datasets/ghibli2real --phase train --light True


2024-07-26 18:48:24.210636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 18:48:24.234093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 18:48:24.240687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 18:48:24.259603: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 18:48:25.767541: W tensorflow/compiler/tf2

In [23]:
!docker pull tensorflow/tensorflow:1.15.0-gpu  # Use -gpu if you need GPU support

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [24]:
!docker run -it --rm -p 8888:8888 tensorflow/tensorflow:1.15.0-gpu  # Adjust port as needed

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [32]:
# prompt: load my datasets for trading from datasets/ghibli2real

# Assuming your datasets are image folders within 'datasets/ghibli2real'
trainA_dataset = datasets.ImageFolder('/content/pytorch-CycleGAN-and-pix2pix/datasets/ghibli2real/trainA', transform=transform)
trainB_dataset = datasets.ImageFolder('/content/pytorch-CycleGAN-and-pix2pix/datasets/ghibli2real/trainB', transform=transform)

# Create dataloaders
trainA_loader = DataLoader(trainA_dataset, batch_size=1, shuffle=True)  # Adjust batch size as needed
trainB_loader = DataLoader(trainB_dataset, batch_size=1, shuffle=True)


FileNotFoundError: Couldn't find any class folder in /content/pytorch-CycleGAN-and-pix2pix/datasets/ghibli2real/trainA.

In [35]:
# prompt: load my datasets for trading from datasets/ghibli2real

# Assuming your datasets are image folders within 'datasets/ghibli2real'
# Verify that this path is correct and that 'trainA' contains subfolders for each class
trainA_dataset = datasets.ImageFolder('/content/pytorch-CycleGAN-and-pix2pix/datasets/ghibli2real/', transform=transform , allow_empty=True)

# Create dataloaders
trainA_loader = DataLoader(trainA_dataset, batch_size=1, shuffle=True)  # Adjust batch size as needed


In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CycleGANGenerator(nn.Module):
    def __init__(self, input_nc, output_nc):
        super(CycleGANGenerator, self).__init__()

        # Encoder
        self.conv1 = nn.Conv2d(input_nc, 64, kernel_size=7, stride=1, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(True)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU(True)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU(True)

        # Bottleneck
        self.residual_blocks = nn.Sequential(*[ResidualBlock(256) for _ in range(9)])

        # Decoder
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU(True)

        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        self.relu5 = nn.ReLU(True)

        self.deconv3 = nn.ConvTranspose2d(64, 3, kernel_size=7, stride=1, padding=3)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))

        x = self.residual_blocks(x)

        x = self.relu4(self.bn4(self.deconv1(x)))
        x = self.relu5(self.bn5(self.deconv2(x)))

        x = self.tanh(self.deconv3(x))

        return x

class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(dim, dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(dim)
        self.relu = nn.ReLU(True)
        self.conv2 = nn.Conv2d(dim, dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(dim)

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        out += identity
        return out


In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CycleGANDiscriminator(nn.Module):
    def __init__(self, input_nc):
        super(CycleGANDiscriminator, self).__init__()

        self.conv1 = nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1)
        self.leaky_relu1 = nn.LeakyReLU(0.2, inplace=False) # Change inplace to False

        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.leaky_relu2 = nn.LeakyReLU(0.2, inplace=False) # Change inplace to False

        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.leaky_relu3 = nn.LeakyReLU(0.2, inplace=False) # Change inplace to False

        self.conv4 = nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1)
        self.leaky_relu4 = nn.LeakyReLU(0.2, inplace=False) # Change inplace to False

        self.conv5 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)

    def forward(self, x):
        x = self.leaky_relu1(self.conv1(x))
        x = self.leaky_relu2(self.conv2(x))
        x = self.leaky_relu3(self.conv3(x))
        x = self.leaky_relu4(self.conv4(x))

        x = self.conv5(x)

        return x

In [64]:
num_epochs = 100

# Hyperparameters
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
lambda_identity = 0.5
lambda_cycle = 10

# Create model instances
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator_G = CycleGANGenerator(input_nc=3, output_nc=3).to(device)
generator_F = CycleGANGenerator(input_nc=3, output_nc=3).to(device)
discriminator_X = CycleGANDiscriminator(input_nc=3).to(device)
discriminator_Y = CycleGANDiscriminator(input_nc=3).to(device)

# Loss functions
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizers
optimizer_G = optim.Adam(itertools.chain(generator_G.parameters(), generator_F.parameters()), lr=lr, betas=(beta1, beta2))
optimizer_D_X = optim.Adam(discriminator_X.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_D_Y = optim.Adam(discriminator_Y.parameters(), lr=lr, betas=(beta1, beta2))



# Training loop
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(trainA_loader):
        print(f"Epoch: {epoch}, Batch: {i}")  # For debugging

        # Separate images into real and anime
        real_A = images[:len(images) // 2].to(device)
        real_B = images[len(images) // 2:].to(device)

        # Generate fake images
        fake_B = generator_G(real_A)
        fake_A = generator_F(real_B)

        # Reconstruct images
        recov_A = generator_F(fake_B)
        recov_B = generator_G(fake_A)

        # Identity loss
        identity_A = generator_F(real_A)
        identity_B = generator_G(real_B)

        # Train discriminators
        optimizer_D_X.zero_grad()
        # Real images
        pred_real_A = discriminator_X(real_A)
        loss_D_real_A = criterion_GAN(pred_real_A, torch.ones_like(pred_real_A))
        # Fake images
        pred_fake_A = discriminator_X(fake_A.detach())
        loss_D_fake_A = criterion_GAN(pred_fake_A, torch.zeros_like(pred_fake_A))
        # Total discriminator loss
        loss_D_A = (loss_D_real_A + loss_D_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_X.step()

        optimizer_D_Y.zero_grad()
        # Real images
        pred_real_B = discriminator_Y(real_B)
        loss_D_real_B = criterion_GAN(pred_real_B, torch.ones_like(pred_real_B))
        # Fake images
        pred_fake_B = discriminator_Y(fake_B.detach())
        loss_D_fake_B = criterion_GAN(pred_fake_B, torch.zeros_like(pred_fake_B))
        # Total discriminator loss
        loss_D_B = (loss_D_real_B + loss_D_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_Y.step()

        # Train generators
        optimizer_G.zero_grad()
        # Adversarial loss
        pred_fake_A = discriminator_X(fake_A)
        loss_G_A = criterion_GAN(pred_fake_A, torch.ones_like(pred_fake_A))
        pred_fake_B = discriminator_Y(fake_B)
        loss_G_B = criterion_GAN(pred_fake_B, torch.ones_like(pred_fake_B))
        # Cycle consistency loss
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)
        # Identity loss
        loss_identity_A = criterion_identity(identity_A, real_A)
        loss_identity_B = criterion_identity(identity_B, real_B)
        # Total generator loss
        loss_G = loss_G_A + loss_G_B + lambda_cycle * (loss_cycle_A + loss_cycle_B) + lambda_identity * (loss_identity_A + loss_identity_B)

        # Check for NaN values
        if torch.isnan(loss_G) or torch.isnan(loss_D_A) or torch.isnan(loss_D_B):
            print("NaN detected in losses")
            break

        loss_G.backward()
        optimizer_G.step()

        # Print losses or save samples as needed
        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(trainA_loader)}] "
                  f"Loss_G: {loss_G.item()}, Loss_D_A: {loss_D_A.item()}, Loss_D_B: {loss_D_B.item()}")
torch.save(generator_G.state_dict(), 'generator_G.pth')
torch.save(generator_F.state_dict(), 'generator_F.pth')
torch.save(discriminator_X.state_dict(), 'discriminator_X.pth')
torch.save(discriminator_Y.state_dict(), 'discriminator_Y.pth')

Epoch: 0, Batch: 0
NaN detected in losses
Epoch: 1, Batch: 0
NaN detected in losses
Epoch: 2, Batch: 0
NaN detected in losses
Epoch: 3, Batch: 0
NaN detected in losses
Epoch: 4, Batch: 0
NaN detected in losses
Epoch: 5, Batch: 0
NaN detected in losses
Epoch: 6, Batch: 0
NaN detected in losses
Epoch: 7, Batch: 0
NaN detected in losses
Epoch: 8, Batch: 0
NaN detected in losses
Epoch: 9, Batch: 0
NaN detected in losses
Epoch: 10, Batch: 0
NaN detected in losses
Epoch: 11, Batch: 0
NaN detected in losses
Epoch: 12, Batch: 0
NaN detected in losses
Epoch: 13, Batch: 0
NaN detected in losses
Epoch: 14, Batch: 0
NaN detected in losses
Epoch: 15, Batch: 0
NaN detected in losses
Epoch: 16, Batch: 0
NaN detected in losses
Epoch: 17, Batch: 0
NaN detected in losses
Epoch: 18, Batch: 0
NaN detected in losses
Epoch: 19, Batch: 0
NaN detected in losses
Epoch: 20, Batch: 0
NaN detected in losses
Epoch: 21, Batch: 0
NaN detected in losses
Epoch: 22, Batch: 0
NaN detected in losses
Epoch: 23, Batch: 0
N

In [68]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Assuming you have a trained generator model saved as 'generator_G.pth'
generator_G_state_dict = torch.load('generator_G.pth')
generator_G = CycleGANGenerator(input_nc=3, output_nc=3) # Recreate the generator instance
generator_G.load_state_dict(generator_G_state_dict) # Load the saved weights

def load_image(image_path):
  image = Image.open(image_path)
  image = transforms.ToTensor()(image).unsqueeze(0)
  return image.to(device)

def save_image(image, path):
  image = image.squeeze(0).cpu().detach().numpy()
  image = image.transpose((1, 2, 0))
  image = (image * 255).astype(np.uint8)
  Image.fromarray(image).save(path)

def convert_and_save(real_image_path, output_path):
  real_image = load_image(real_image_path)
  with torch.no_grad():
    fake_image = generator_G(real_image)
  save_image(fake_image, output_path)



In [71]:
real_image_path = '/content/dataset/trainA/1000523639.jpg'
output_path = '/content/dataset/image.jpg'

convert_and_save(real_image_path, output_path)